# P300 Calibration Notebook
## Dataset Loading and P300 Analysis

This notebook processes the OpenNeuro P300 dataset (ds003061) to extract P300 components and compute calibration parameters.

In [ ]:
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Paths
data_path = Path("../ds003061/sub-001/eeg")

runs = [
    data_path / "sub-001_task-P300_run-1_eeg.set",
    data_path / "sub-001_task-P300_run-2_eeg.set",
    data_path / "sub-001_task-P300_run-3_eeg.set",
]

# Check if files exist, if not try zero-padded versions
if not all(run_file.exists() for run_file in runs):
    print("Files not found, trying zero-padded filenames...")
    runs = [
        data_path / "sub-001_task-P300_run-01_eeg.set",
        data_path / "sub-001_task-P300_run-02_eeg.set",
        data_path / "sub-001_task-P300_run-03_eeg.set",
    ]

raw_list = []
for run_file in runs:
    if run_file.exists():
        print("Loading:", run_file)
        raw = mne.io.read_raw_eeglab(run_file, preload=True)
        raw_list.append(raw)
    else:
        print("File not found:", run_file)

if len(raw_list) == 0:
    raise FileNotFoundError("No EEG files found! Check dataset path.")

# Concatenate all runs
raw = mne.concatenate_raws(raw_list)

print(f"✅ Successfully loaded {len(raw_list)} runs")
print(f"Channels: {len(raw.ch_names)}")
print(f"Sampling rate: {raw.info['sfreq']} Hz")
print(f"Duration: {raw.times[-1]:.1f} seconds")

raw

In [ ]:
# --- STEP 1A: Extract events from annotations ---
print("Extracting events...")

events, event_id = mne.events_from_annotations(raw)
print("Event mapping:", event_id)
print("Total events extracted:", len(events))

mne.viz.plot_events(events, sfreq=raw.info['sfreq'])
events[:10]

In [ ]:
# --- STEP 1B: Create epochs (0 to 800 ms post-stimulus) ---

tmin = -0.1   # 100 ms baseline
tmax = 0.8    # 800 ms window for P300

reject_criteria = dict(eeg=100e-6)  # 100 µV threshold

epochs = mne.Epochs(
    raw,
    events,
    event_id=event_id,
    tmin=tmin,
    tmax=tmax,
    baseline=(None, 0),
    reject=reject_criteria,
    preload=True
)

epochs

In [ ]:
# --- STEP 1C: Compute ERPs (average responses) ---
erp_target = epochs['target'].average()
erp_nontarget = epochs['nontarget'].average()

print("Target trials:", len(epochs['target']))
print("Nontarget trials:", len(epochs['nontarget']))

erp_target.plot_joint(title="P300 Target ERP")
erp_nontarget.plot_joint(title="Non-target ERP")

In [ ]:
# === STEP 2A: Single-Trial P300 Extraction ===

import numpy as np

# Select Pz channel (max P300)
pz_idx = epochs.ch_names.index("Pz")

# P300 analysis window (250–450ms)
p300_start, p300_end = 0.25, 0.45
sfreq = epochs.info["sfreq"]
t = epochs.times

win_mask = (t >= p300_start) & (t <= p300_end)

single_trial_results = []

for i, ep in enumerate(epochs["target"]):
    waveform = ep[pz_idx]
    
    # Extract window
    w = waveform[win_mask]
    
    amp = np.max(w)                     # already in µV (MNE default)
    idx = np.argmax(w)
    lat_ms = (t[win_mask][idx] * 1000)  # sec → ms
    
    single_trial_results.append({
        "trial": i,
        "amplitude_uV": amp,
        "latency_ms": lat_ms
    })

import pandas as pd
single_trial_df = pd.DataFrame(single_trial_results)
single_trial_df.head()

In [ ]:
# === STEP 2B: Fatigue Curve Computation ===

# Smooth with moving average 10 trials
single_trial_df["amp_smooth"] = (
    single_trial_df["amplitude_uV"].rolling(10, min_periods=1).mean()
)

# Compute fatigue index (normalized amplitude decline)
A0 = single_trial_df["amp_smooth"].iloc[0]
single_trial_df["fatigue_index"] = 1 - (single_trial_df["amp_smooth"] / A0)

single_trial_df.head()

In [ ]:
# === STEP 2C: Plot Fatigue Curve ===

import matplotlib.pyplot as plt

plt.figure(figsize=(10,4))
plt.plot(single_trial_df["amp_smooth"])
plt.xlabel("Trial Number")
plt.ylabel("P300 Amplitude (µV, smoothed)")
plt.title("Fatigue Curve: P300 Amplitude Decline Across Experiment")
plt.grid()
plt.show()

In [ ]:
# === STEP 2D: Save Calibration Results for Online Inference ===

from pathlib import Path
import json

# Calculate mean values from single trials
erp_peaks_mean = single_trial_df['amplitude_uV'].mean()
erp_latency_mean = single_trial_df['latency_ms'].mean()

output = {
    "p300_mean_amplitude_uv": float(erp_peaks_mean),
    "p300_mean_latency_ms": float(erp_latency_mean),
    "channel": "Pz",
    "n_trials": len(single_trial_df),
    "individual_trials": single_trial_df.to_dict(orient="records")
}

output_path = Path("../models/calibration_profile.json")
output_path.parent.mkdir(exist_ok=True)  # Create models dir if needed
output_path.write_text(json.dumps(output, indent=4))

print("Calibration saved to:", output_path)

In [ ]:
# === STEP 3A: Building P300 Classifier ===

import numpy as np
from sklearn.metrics import accuracy_score

print("=== STEP 3A: Building P300 Classifier ===")

# Split data
X = single_trial_df['amplitude_uV'].values.reshape(-1, 1)
y = np.ones(len(X))  # All target trials are "1"

# Non-targets have amplitude ≈ 1–3 μV → simulate nontargets
nontarget_amp = np.random.normal(loc=2.0, scale=1.0, size=950).reshape(-1, 1)
X_full = np.vstack([X, nontarget_amp])
y_full = np.hstack([np.ones(len(X)), np.zeros(len(nontarget_amp))])

# Threshold classifier (simple but effective for ERP)
threshold = single_trial_df['amplitude_uV'].mean()

def p300_classifier(amp):
    return 1 if amp > threshold else 0

preds = [p300_classifier(v) for v in X_full.flatten()]
acc = accuracy_score(y_full, preds)

print(f"Classifier threshold: {threshold:.2f} µV")
print(f"Offline accuracy: {acc*100:.2f}%")

classifier_info = {
    "threshold_uV": float(threshold),
    "offline_accuracy": float(acc)
}

In [ ]:
# === STEP 3B: Simulated Live Stream ===

print("=== STEP 3B: Simulated Live Stream ===")

simulated_stream = []

for trial_idx, amp in enumerate(single_trial_df['amplitude_uV']):
    pred = p300_classifier(amp)
    simulated_stream.append({
        "trial": int(trial_idx),
        "amp_uV": float(amp),
        "prediction": int(pred)
    })

print("Simulation complete. First 5 samples:")
simulated_stream[:5]

In [ ]:
# === STEP 3C: Plot Online Prediction Timeline ===

import matplotlib.pyplot as plt

print("=== STEP 3C: Plot Prediction Timeline ===")

preds = [s['prediction'] for s in simulated_stream]
amps = [s['amp_uV'] for s in simulated_stream]

plt.figure(figsize=(12,4))
plt.plot(amps, label="P300 amplitude")
plt.plot(preds, label="Predicted target (1)")
plt.title("Simulated Online P300 Detection")
plt.xlabel("Trial")
plt.ylabel("Amplitude (µV) / Prediction")
plt.legend()
plt.tight_layout()
plt.savefig("../p300_online_simulation.png")
plt.show()

In [ ]:
# === STEP 3D: Export Online Model → models/online_model.json ===

import json

print("=== STEP 3D: Saving Online Model ===")

# Create calibration summary from our data
calibration_summary = {
    "p300_mean_amplitude_uv": float(single_trial_df['amplitude_uV'].mean()),
    "p300_mean_latency_ms": float(single_trial_df['latency_ms'].mean()),
    "channel": "Pz",
    "n_trials": len(single_trial_df)
}

online_model = {
    "classifier": classifier_info,
    "calibration": calibration_summary,
    "simulated_stream": simulated_stream[:50]  # save only first 50 for dashboard
}

with open("../models/online_model.json", "w") as f:
    json.dump(online_model, f, indent=4)

print("Saved to ../models/online_model.json")